<a href="https://colab.research.google.com/github/AIkikr/code-translation-pipeline/blob/main/400G_phi_distill_SFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# ファイルをアップロード
uploaded1 = files.upload()



Saving 400G_IEEE補強_clean.jsonl to 400G_IEEE補強_clean.jsonl


In [ ]:
### Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# python 3.10.12
!pip install -U pip
!pip install -U transformers
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U datasets
!pip install -U peft
!pip install -U trl
!pip install -U wandb
!pip install ipywidgets --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 144.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.28.1
    Uninstalling huggingface-hub-0.28.1:
      Successfully uninstalled huggingface-hub-0.28.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 150.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 179.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 173.4 MB/s eta 0:00:00
   ━━━━━━

In [ ]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
)
import os, torch, gc
from datasets import load_dataset
import bitsandbytes as bnb
from trl import SFTTrainer

from transformers import DataCollatorForLanguageModeling


In [ ]:
from google.colab import drive, files
from getpass import getpass
import os
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# トークンを環境変数に設定
os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token

# CLIにログイン
!huggingface-cli login --token $HUGGINGFACE_HUB_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `HF_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `HF_TOKEN`


In [ ]:
from IPython.display import Javascript
#JavaScriptコードを埋め込む
keep_alive_code = """
function keepColabAlive() {
    setInterval(() => {
        document.querySelector("colab-toolbar-button#connect").click();
    }, 600000); // 10分ごとにクリック
}
keepColabAlive();
"""

# JavaScriptを実行
display(Javascript(keep_alive_code))

<IPython.core.display.Javascript object>

In [ ]:
### 1) Prepare dataset: JSONL with instruction/output
from datasets import load_dataset

data_files = {'train': '400G_IEEE補強_clean.jsonl'}
dataset = load_dataset('json', data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
### 2) Load tokenizer & set padding_side for decoder
from transformers import AutoTokenizer
base_model_id = 'microsoft/phi-4'
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
# Ensure pad token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
# Left padding for decoder-only
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.25M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

In [ ]:
### 3) Formatting & tokenization function (phi-4 recommended template)
SYSTEM = '<|system|>あなたは400G EthernetとOTNネットワーク設計の専門家です。論理的かつ論文調で回答してください。<|end|>'
USER_TMPL = '<|user|>{instr}<|end|>'
ASSIST = '<|assistant|>'

def tokenize_example(ex):
    # build prompt+answer string
    prompt = "\n".join([
        'text',
        SYSTEM,
        USER_TMPL.format(instr=ex['instruction']),
        ASSIST,
        ex['output'],
        '<|end|>'
    ])
    # tokenize
    return tokenizer(
        prompt,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors=None
    )

# map train split
tokenized = dataset['train'].map(
    tokenize_example,
    batched=False,
    remove_columns=dataset['train'].column_names
)

tokenized.set_format(type='torch', columns=['input_ids','attention_mask'])

Map:   0%|          | 0/14307 [00:00<?, ? examples/s]

In [ ]:
### 4) Load base model + distilled adapter
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, LoraConfig, get_peft_model

# quant config: reuse 4bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4',
)
# load base phi-4
base = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True
)
# attach distilled adapter
distill_adapter = 'AkiK/phi-4-4bit-Q-lora-kura-distill-simple'
model = PeftModel.from_pretrained(base, distill_adapter)
# merge for efficiency
model = model.merge_and_unload()

config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/871 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/29.5M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
### 5) Add new LoRA for SFT
# collect target modules (optional: reuse same list as before)
import bitsandbytes as bnb

def find_linear_modules(m):
    cls = bnb.nn.Linear8bitLt
    names = set()
    for n, mod in m.named_modules():
        if isinstance(mod, cls):
            parts = n.split('.')
            names.add(parts[-1])
    names.discard('lm_head')
    return list(names)

target_modules = find_linear_modules(model)

peft_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.1,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=target_modules
)
model = get_peft_model(model, peft_config)

In [ ]:
### 6) TrainingArguments & Trainer
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer, padding='longest')

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/oki_llama_project_2/checkpoints',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim='paged_adamw_8bit',
    num_train_epochs=3,
    learning_rate=3e-5,
    logging_steps=20,
    save_steps=200,
    save_total_limit=3,
    fp16=True,
    bf16=False,
    group_by_length=True,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=data_collator
)


In [ ]:
### 7) Disable cache and start training
model.config.use_cache = False
trainer.train()

In [ ]:
!pip install tqdm
from tqdm import tqdm

In [ ]:
# Google Driveのマウント
from google.colab import drive
import os
from google.colab import userdata

# ドライブをマウント
drive.mount('/content/drive', force_remount=True)


# Hugging Faceトークンの取得
try:
    hf_token = userdata.get('HF_TOKEN')
    os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token
except Exception as e:
    print(f"トークン取得エラー: {e}")
    # 手動入力のバックアップ方法
    from getpass import getpass
    hf_token = getpass("Hugging Faceトークンを入力してください: ")

# CLIにログイン
!huggingface-cli login --token $HUGGINGFACE_HUB_TOKEN

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
new_model_id = "AkiK/phi-4-4bit-Q-lora-kura-distill-simple-SFT"  # 好きなモデルIDに変更

# 環境変数からトークンを取得
hf_token = os.environ.get('HUGGINGFACE_HUB_TOKEN')

# トークンを使ってモデルとトークナイザーをプッシュ
model.push_to_hub(new_model_id, token=hf_token, private=True)
tokenizer.push_to_hub(new_model_id, token=hf_token, private=True)

In [ ]:
pip install safetensors datasets

In [ ]:
!pip install sacrebleu

In [ ]:
del model, tokenizer
torch.cuda.empty_cache()

In [ ]:
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, PeftModel
from torch.nn import KLDivLoss
import torch.nn.functional as F

In [ ]:
import json
import random
import torch
import pandas as pd
import matplotlib.pyplot as plt
from sacrebleu import corpus_bleu
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1) Quant 設定は同じものを再利用
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

# 2) ベースモデル（量子化済み）を先に読み込む
base_model_id = "microsoft/phi-4"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# 3) そこに Hub のアダプタをマージ
adapter_repo = "AkiK/phi-4-4bit-Q-lora-kura-distill-simple"
model = PeftModel.from_pretrained(model, adapter_repo)
# （必要ならマージして１つの重みファイルに統合）
model = model.merge_and_unload()

model.eval()